# Intro

This jupyter notebook can calculate optimal defense EVs for a pokemon, under a number of simplifying assumptions. If you want to skip straight to the calculator, navigate to the "Using the Widget" section two cells down

## Optimal EV calculation

When we give EVs to a defensive pokemon, we want to maximize the amount of damage it can take before fainting

One way to optimize EVs is to do calculations with specific attacks (e.g. jolly excadrill max-steelspike), and tailor EVs to survive them. However, this is difficult to do if you are not sure what kind of attacks to expect

Without specific attacks to base EVs on, we use heuristics. Most people have the general sense that adding to HP is the best, and then the optimal spread across Def/SpD depends on what kind of hits the pokemon is expecting to take. Can we quantify these instincts in a rigorous way?

Simplifying the damage formula a bit, we can think of damage dealt as "offense" divided by "defense". Offense is a function of the attacking pokemon's attack, move power, type effectiveness, and other damage modifiers. Defense is the corresponding defense based on the type of attack, either physical or special defense

Assume that a pokemon takes $L$ offense of which ratio $P$ is physical and $1-P$ is special (e.g. if a pokemon takes 400 offense of physical damage and 600 of special damage, $L$ = 1000 and $P$ = 0.4). It will take $\frac{L * P}{Def} + \frac{L*(1-P)}{SpD}$ damage. If this is at or above the pokemon's HP, the pokemon will faint. We want to maximize the value of L needed to faint the pokemon,  so we can define our problem as $max(L)$ where $\frac{L * P}{Def} + \frac{L*(1-P)}{SpD} = HP$ and $HP$, $Def$, and $SpD$ are constrained by the limits of EV and IV training. We can think of $L$ as a measure of longevity. The higher it is, the more offense a pokemon can take before fainting

Unfortunately, the constraint is non-convex and thus the problem is difficult to optimize explicitly. Fortunately, the solution space is rather small, and a computer can quickly optimize this function by checking every sensible input

If we solve for L as a function of $HP$, $Def$, and $SpD$, we can query the function for different EV and nature values and see which combination yields the highest L

\begin{equation}
\frac{L*P}{Def} + \frac{L*(1-P)}{SpD} = HP \\[10pt]
\frac{L*P*SpD + L*(1-P)*Def}{Def*SpD} = HP \\[10pt]
L*P*SpD + L*(1-P)*Def = HP*Def*SpD \\[10pt]
L*(P*SpD + (1-P)*Def) = HP*Def*SpD \\[10pt]
L = \frac{HP*Def*SpD}{P*SpD + (1-P)*Def} \\[10pt]
\end{equation}

The calculate function that you can call below iterates through sensible EV and nature choices, and spits out the set that maximizes L. Be sure to follow the instructions listed in the cell below it

In [1]:
import ipywidgets as widgets
from IPython.display import display

individual_ev_max = 252

def optimize_evs(hp, defense, special_defense, ev_max, individual_ev_max, physical_fraction, nature):
    
    best_damage_taken = 0

    for hp_investment in range(0,int((min(individual_ev_max,ev_max)+4)/8 + 1)):
        hp_evs = max(hp_investment*8-4,0)

        for defense_investment in range(0, int((min(individual_ev_max,ev_max-hp_evs)+4)/8 + 1)):
            
            defense_evs = max(defense_investment*8-4,0)
            
            special_defense_evs = max(min(individual_ev_max,ev_max-hp_evs-defense_evs),0)
            special_defense_investment = int((special_defense_evs+4)/8)

            hp_total = hp + hp_investment
            
            for defense_nature in [True, False] if nature else [False]:
                
                defense_total = defense + defense_investment
                special_defense_total = special_defense + special_defense_investment 
                
                if defense_nature:
                    defense_total = int(defense_total*1.1)
                elif nature:
                    special_defense_total = int(special_defense_total*1.1)
                
                denom = ((1-physical_fraction)*defense_total +(physical_fraction)*special_defense_total)
                if (denom > 0):
                    damage_taken = hp_total*defense_total*special_defense_total/denom

                    if damage_taken >= best_damage_taken:
                        best_damage_taken = damage_taken

                        best_defense_evs = defense_evs
                        best_special_defense_evs = special_defense_evs
                        best_hp_evs = hp_evs
                        best_nature = 'D' if defense_nature else 'SpD' if nature else None
            
    return best_hp_evs, best_defense_evs, best_special_defense_evs, best_nature

style = {'description_width': 'initial'}

hp = widgets.BoundedIntText(min = 0,
                            max = 1000,
                            step = 4,
                            value=160,
                            description='HP:',
                            disabled=False,
                            style = style
                        )
defense = widgets.BoundedIntText(min = 0,
                            max = 1000,
                            step = 4,
                            value=110,
                            description='Defense:',
                            disabled=False,
                            style = style

                        )
special_defense = widgets.BoundedIntText(
                            min = 0,
                            max = 1000,
                            step = 4,
                            value=115,
                            description='Special Defense:',
                            disabled=False,
                            style = style

                        )

ev_max = widgets.BoundedIntText(
                            value=508,
                            min = 0,
                            max = 508,
                            step = 4,
                            description='EVs available:',
                            disabled=False,
                            style = style

                        )

physical_fraction = widgets.FloatSlider(
                            min=0,
                            max=1,
                            step = 0.01,
                            readout_format='.2f',
                            value = 0.6,
                            description='Physical Fraction (P):',
                            disabled=False,
                            style = style
                        )

nature = widgets.Checkbox(
                            value= True,
                            description='Use + Defense/Special Defense Nature',
                            disabled=False,
                            style = style

                        )

button = widgets.Button(description = 'Calculate',
                       style = style)

hp_output = widgets.Text(description = 'HP EVs:'
                        ,disabled = True
                        ,style = style)

defense_output = widgets.Text(description = 'Defense EVs:'
                        ,disabled = True
                        ,style = style)

special_defense_output = widgets.Text(description = 'Special Defense EVs:'
                        ,disabled = True
                        ,style = style)

def on_button_clicked(b):
    
    hp_val, def_val, spd_val, n = optimize_evs(hp.value
                               ,defense.value
                               ,special_defense.value
                               ,ev_max.value
                               ,individual_ev_max
                               ,physical_fraction.value
                               ,nature.value
                )
    hp_output.value = str(hp_val)
    defense_output.value = str(def_val) + '+' if n == 'D' else str(def_val)
    special_defense_output.value = str(spd_val) + '+' if n == 'SpD' else str(spd_val)

button.on_click(on_button_clicked)

display(hp)
display(defense)
display(special_defense)
display(ev_max)
display(nature)
display(physical_fraction)
display(button)
display(hp_output)
display(defense_output)
display(special_defense_output)

BoundedIntText(value=160, description='HP:', max=1000, step=4, style=DescriptionStyle(description_width='initi…

BoundedIntText(value=110, description='Defense:', max=1000, step=4, style=DescriptionStyle(description_width='…

BoundedIntText(value=115, description='Special Defense:', max=1000, step=4, style=DescriptionStyle(description…

BoundedIntText(value=508, description='EVs available:', max=508, step=4, style=DescriptionStyle(description_wi…

Checkbox(value=True, description='Use + Defense/Special Defense Nature', style=DescriptionStyle(description_wi…

FloatSlider(value=0.6, description='Physical Fraction (P):', max=1.0, step=0.01, style=SliderStyle(description…

Button(description='Calculate', style=ButtonStyle())

Text(value='', description='HP EVs:', disabled=True, style=DescriptionStyle(description_width='initial'))

Text(value='', description='Defense EVs:', disabled=True, style=DescriptionStyle(description_width='initial'))

Text(value='', description='Special Defense EVs:', disabled=True, style=DescriptionStyle(description_width='in…

## Using the Widget 

IMPORTANT INSTRUCTIONS: 
- You need to wait until the page fully loads to use the calculator. If the page is still loading, you will see spinning circles at the top
- To use the EV optimizer widget above, select the code cell above it and press SHIFT-ENTER. A number of controllable widgets should appear, along with a button that says 'calculate'. Hit calculate to run the optimization function

Widget fields
- HP, Defense, and Special Defense are the pokemon's original values of the statistic, before any EVs are added
- EVs available is how many EVs you can devote to defense
    - In many circumstances, this number will be 508. However if you want to EV for another stat like attack, you need to subtract that amount from 508 to get the remaining EVs. For example if I want to give my porygon2 100 EVs of special attack, I have 408 EVs remaining
- Check the 'Use + Defense/Special Defense Nature' if you can use a nature to improve defenses, like bold (+ Def,- Atk) or sassy (+ SpD, - Spe)
- The physical fraction (P) is the fraction of offense which is physical
    - This factors in typing. For example if you expect 50% physical super-effective physical attacks and 50% normally effective special attacks, in total physical attacks account for 67% of the total offense ($\frac{0.5*2}{0.5*2+0.5}$)
    - PS: You can control the Physical Fraction ratio manually by right-clicking the number on the right, typing a new fraction, and hitting enter
- The EV fields will be populated after hitting 'calculate'. These contain the calculated optimal EV values. If a value is followed by '+', that means the stat is also boosted by a nature

Caveats
- The calculate function only returns one optimal set- there may be other(s) which are equally optimal but not shown
- This calculator is for vgc and pokemon are assumed to be lvl. 50

**************

## Example: Porygon2

The default values are for my porygon2. With perfect IV's and no EV's, my porygon2 has 160 HP, 110 Defense, and 115 Special defense (we don't have to worry about eviolite because adding 50% to Def and SpD will allow porygon to take 50% more offense no matter what). I expect it to take 60% of opposing offense from physical attacks, because porygon2 is weak to fighting and fighting attacks are mostly physical 

Hit the 'calculate' button with the default parameters to get a spread for my porygon2. the spread should be 252 HP, 236+ Def , and 20 SpD. This makes intuitive sense because we expect more physical attacks and porygon2's defense starts lower than its special defense 

Now if I feel like it, I can look up specific calculations and make adjustments. For example, I am curious about porygon-Z. I see that a modest life orb porygon-Z has a 25% chance of 0HKOing this porygon. Porygon-Z is 8% of the metagame, and roughly half are modest, so this will be relevant 4% of the time and result in an 0HKO 1% of the time if PZ always attacks P2. Adding 16 SpD EVs would reduce to chance of an 0HKO to 12.5%, so I might change the spread manually to (252,220,36,' + Defense Nature') if I am concerned about this

*****************

## Insights

### The importance of rounding

If you play around with the numbers a bit, you might notice the distribution changing a surprising amount with little change in input variables. For example, try inputting (160,110,115,332,0.5) and (160,110,115,340,0.5). The result is (252,20+,60) for the first and (252,76+,12) for the second. Why would the code change the distribution so drastically with one extra stat point to invest? 

The reason is that 76+ defense generates a defense stat of exactly 132 before rounding, which is efficient because nothing is lost from rounding. That benefit can be enough to entice the optimizer to change the spread significantly when it becomes feasible without sacrificing too much special defense investment

### Combining defenses with the harmonic mean

It is useful to evaluate a pokemon's defenses by synthesizing the SpD and Def stats into an "overall defense" metric. Taking the average might seem like a natural way to combine the two. However, it doesn't reflect the reality that the pokemon will faint more quickly by the path of least resistance. For example, a pokemon with 1 D and 1,000,000 SpD will faint quickly to a combined barrage of special and physical attacks, while a pokemon with 1,000 D and 1,000 SpD will take many hits before fainting. The math above points us to a different way of combining defenses, which accurately measures its effect on longevity under a mixed set of attacks. Assuming $P = 0.5$:

\begin{equation}
L = \frac{HP*Def*SpD}{P*SpD + (1-P)*Def} \\[10pt]
L = \frac{HP*Def*SpD}{\frac{Def}{2} + \frac{SpD}{2}} \\[10pt]
L = HP * 2* \frac{Def*SpD}{Def + SpD} \\[10pt]
L = HP * \frac{2}{\frac{1}{Def} + \frac{1}{SpD}} \\[10pt]
\end{equation}

$\frac{2}{\frac{1}{Def} + \frac{1}{SpD}}$ is called the harmonic mean of $Def$ and $SpD$. This is the sensible way of combining the two, because we can see it has a linear effect on longevity. If we want to add $P$ back to the definition, it creates a weighted harmonic mean which looks like this

\begin{equation}
\frac{1}{\frac{1-P}{SpD} + \frac{P}{Def}}
\end{equation}

### A connection to circuits

If you have studied electricity, you may recognize the harmonic mean from the formula for resistance across a parallel circuit: $R_t = \frac{1}{\frac{1}{R_1} + \frac{1}{R_2}}$ (The resistance of parallel resistors is the harmonic mean of individual resistances divided by the number of resistors, which is why it has no $2$ in the numerator and is not exactly the same as our defense formula. Still, it is quite similar). Thinking of $D_t$ as the "total defense" such that $HP * D_t = L$, we can derive formulas for $R_t$ and $D_t$ with the same steps. Note that HP here means "HP that needs to be lost before fainting", so we can break it down into components $HP_{Def}$ and $HP_{SpD}$ which represent the HP that our pokemon will lose in both categories before fainting, again assuming $P = 0.5$

For those unfamiliar with E&M, $I$ is a symbol for current and $V$ is a symbol for Voltage. They are linked by Ohm's law, which states $V = IR$ 

\begin{align}
I = I_1 + I_2 && HP = HP_{Def} + HP_{SpD} \\[10pt]
I = \frac{V}{R_1} +  \frac{V}{R_2} && HP = \frac{L}{2*D} +  \frac{L}{2*SpD}\\[10pt]
I = V * (\frac{1}{R_1} +  \frac{1}{R_2}) && HP = L*(\frac{1}{2*D} +  \frac{1}{2*SpD})\\[10pt]
\frac{I}{V} = (\frac{1}{R_1} +  \frac{1}{R_2}) && \frac{HP}{L} = (\frac{1}{2*D} +  \frac{1}{2*SpD}) \\[10pt]
\frac{V}{I} = \frac{1}{\frac{1}{R_1} +  \frac{1}{R_2}} && \frac{L}{HP} = \frac{1}{\frac{1}{2*D} +  \frac{1}{2*SpD}} \\[10pt]
R_t = \frac{1}{\frac{1}{R_1} + \frac{1}{R_2}} && D_t = \frac{2}{\frac{1}{D} + \frac{1}{SpD}}
\end{align}